# Preparation

## Imports

In [729]:
import pmp_functions_v5 as pmp
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

## Functions

In [730]:
def combine_strategy_results(dfs, weights):
    """
    Combine multiple strategy result DataFrames into a single meta-portfolio.

    Parameters
    ----------
    dfs : list of pd.DataFrame
        List of backtest result tables (each must contain ret_net, turnover, tcost, ret_rf, ret_bm, etc.)

    weights : list of float
        Strategy-level weights (must sum to 1.0)

    Returns
    -------
    final_results : pd.DataFrame
        Combined backtest results with aggregated performance and exposure.
    """

    import pandas as pd

    if abs(sum(weights) - 1.0) > 1e-8:
        raise ValueError("Strategy weights must sum to 1.0")

    # --------------------------------------------------------
    # 1) Align on common index (intersection)
    # --------------------------------------------------------
    common_index = dfs[0].index
    for df in dfs[1:]:
        common_index = common_index.intersection(df.index)

    dfs_aligned = [df.reindex(common_index) for df in dfs]

    # --------------------------------------------------------
    # 2) Combine performance metrics
    # --------------------------------------------------------
    ret_net_combined   = sum(w * df["ret_net"]   for w, df in zip(weights, dfs_aligned))
    ret_gross_combined = sum(w * df["ret_gross"] for w, df in zip(weights, dfs_aligned))
    ret_bm_combined    = sum(w * df["ret_bm"]    for w, df in zip(weights, dfs_aligned))
    turnover_combined  = sum(w * df["turnover"]  for w, df in zip(weights, dfs_aligned))
    tcost_combined     = sum(w * df["tcost"]     for w, df in zip(weights, dfs_aligned))
    ret_rf_combined    = sum(w * df["ret_rf"]    for w, df in zip(weights, dfs_aligned))

    # Performance-only DataFrame
    final_results = pd.DataFrame({
        "ret_net": ret_net_combined,
        "ret_gross": ret_gross_combined,
        "ret_bm": ret_bm_combined,
        "turnover": turnover_combined,
        "tcost": tcost_combined,
        "ret_rf": ret_rf_combined,
    })

    # --------------------------------------------------------
    # 3) Collect all weight columns across all strategies
    # --------------------------------------------------------
    region_cols = sorted({
        col for df in dfs_aligned for col in df.columns if col.startswith("w_")
    })

    # --------------------------------------------------------
    # 4) Ensure all strategies have all weight columns, fill missing with 0
    # --------------------------------------------------------
    for i, df in enumerate(dfs_aligned):
        for reg in region_cols:
            if reg not in df.columns:
                dfs_aligned[i][reg] = 0.0

    # --------------------------------------------------------
    # 5) Combine region weights using strategy weights
    # --------------------------------------------------------
    for reg in region_cols:
        final_results[reg] = sum(w * df[reg] for w, df in zip(weights, dfs_aligned))

    return final_results


In [731]:
def apply_fx_hedging(backtest_df, hedge_ratio, hedge_ratio_short, hedge_returns):
    """
    Adds FX hedging P&L on top of an existing backtest.
    
    - USD is treated as the base currency and will NOT be hedged.
    """

    df = backtest_df.copy()

    # Extract region names from weight columns
    regions = [col.replace("w_", "") for col in df.columns if col.startswith("w_")]

    # Remove USD from hedging universe
    regions = [r for r in regions if r != "US"]

    # Align hedge data with Date index
    hedge_ratio = hedge_ratio.reindex(df.index)
    hedge_ratio_short = hedge_ratio_short.reindex(df.index)
    hedge_returns = hedge_returns.reindex(df.index)

    hedge_pnl = pd.Series(0.0, index=df.index)

    for region in regions:

        w_col = f"w_{region}"

        # Check hedge ratio availability (should match non-US regions)
        if region not in hedge_ratio.columns:
            print(f"⚠ Region {region} not in hedge_ratio — skipping.")
            continue

        # Choose long vs short hedge ratio depending on sign of weight
        long_mask = df[w_col] >= 0
        short_mask = df[w_col] < 0

        hedge_mult = pd.Series(0.0, index=df.index)
        hedge_mult[long_mask] = hedge_ratio[region][long_mask]
        hedge_mult[short_mask] = hedge_ratio_short[region][short_mask]

        # Check hedge return availability
        if region not in hedge_returns.columns:
            print(f"⚠ Region {region} not in hedge_returns — skipping.")
            continue

        # Hedge contribution
        contrib = df[w_col] * hedge_mult * hedge_returns[region]

        hedge_pnl += contrib

    # Add hedged P&L to ret_net
    df["ret_net"] = df["ret_net"] + hedge_pnl

    return df

In [732]:
def combine_macro_weights(dfs, weights):
    """
    Combine multiple macro-strategy weight tables into a single weighted average.

    Parameters
    ----------
    dfs : list of DataFrame
        Weight tables (w_AU, w_CH, ...)
    weights : list of float
        Strategy weights, must sum to 1.0

    Returns
    -------
    DataFrame
        Combined weighted sum of all weights
    """
    import pandas as pd
    import numpy as np

    # Align all to the same index
    common_index = dfs[0].index
    for df in dfs[1:]:
        common_index = common_index.intersection(df.index)

    dfs_aligned = [df.reindex(common_index) for df in dfs]

    # Identify region columns
    region_cols = [c for c in dfs_aligned[0].columns if c.startswith("w_")]

    # Weighted sum
    combined = pd.DataFrame(index=common_index)

    for col in region_cols:
        combined[col] = sum(w * df[col] for w, df in zip(weights, dfs_aligned))

    return combined


In [733]:
def combine_two_weight_tables(df1, df2, weight1=0.5, weight2=0.5):
    """
    Combine two weight tables (e.g., trend + macro) using weighted average.
    Missing regions in either table are treated as 0.
    """
    import pandas as pd

    # 1. Align indices
    idx = df1.index.union(df2.index)
    df1a = df1.reindex(idx)
    df2a = df2.reindex(idx)

    # 2. Align columns (regions)
    cols = sorted(set(df1a.columns).union(df2a.columns))
    df1a = df1a.reindex(columns=cols).fillna(0.0)
    df2a = df2a.reindex(columns=cols).fillna(0.0)

    # 3. Weighted combination
    combined = weight1 * df1a + weight2 * df2a

    return combined


## Data

### Strategies

In [734]:
# --- Fundamental Data ---
fundamental_equity = pd.read_csv("../Results/fundamental_equity.csv", index_col=0, parse_dates=True)
fundamental_equity.rename(columns={"w_GB": "w_UK"}, inplace=True)

# --- Macro Data ---
macroRS_equity = pd.read_csv("../Results/macroRS_equity.csv", index_col=0, parse_dates=True)
macroRS_bond = pd.read_csv("../Results/macroRS_bond.csv", index_col=0, parse_dates=True)
macroRS_rates = pd.read_csv("../Results/macroRS_rates.csv", index_col=0, parse_dates=True)
macroRS_fx = pd.read_csv("../Results/macroRS_fx.csv", index_col=0, parse_dates=True)

macroIT_equity = pd.read_csv("../Results/macroIT_equity.csv", index_col=0, parse_dates=True)
macroIT_bond = pd.read_csv("../Results/macroIT_bond.csv", index_col=0, parse_dates=True)
macroIT_rates = pd.read_csv("../Results/macroIT_rates.csv", index_col=0, parse_dates=True)
macroIT_fx = pd.read_csv("../Results/macroIT_fx.csv", index_col=0, parse_dates=True)

macroMP_equity = pd.read_csv("../Results/macroMP_equity.csv", index_col=0, parse_dates=True)
macroMP_bond = pd.read_csv("../Results/macroMP_bond.csv", index_col=0, parse_dates=True)
macroMP_rates = pd.read_csv("../Results/macroMP_rates.csv", index_col=0, parse_dates=True)
macroMP_fx = pd.read_csv("../Results/macroMP_fx.csv", index_col=0, parse_dates=True)    

macroBC_equity = pd.read_csv("../Results/macroBC_equity.csv", index_col=0, parse_dates=True)
macroBC_bond = pd.read_csv("../Results/macroBC_bond.csv", index_col=0, parse_dates=True)
macroBC_rates = pd.read_csv("../Results/macroBC_rates.csv", index_col=0, parse_dates=True)
macroBC_fx = pd.read_csv("../Results/macroBC_fx.csv", index_col=0, parse_dates=True)


# --- Trend Following Data ---
trend_equity = pd.read_csv("../Results/trend_equity.csv", index_col=0, parse_dates=True)
trend_bond = pd.read_csv("../Results/trend_bond.csv", index_col=0, parse_dates=True)
trend_rates = pd.read_csv("../Results/trend_rates.csv", index_col=0, parse_dates=True)
trend_fx = pd.read_csv("../Results/trend_fx.csv", index_col=0, parse_dates=True)

In [735]:
def extract_clean_weights(df):
    # 1) Nur die weight-Spalten nehmen
    weight_cols = [c for c in df.columns if c.startswith("w_")]
    
    # 2) Gewichtstabelle extrahieren
    w = df[weight_cols].copy()
    
    # 3) Prefix "w_" entfernen
    w.columns = [c.replace("w_", "") for c in weight_cols]
    
    return w

# Clean weights for both strategies
fundamental_equity = extract_clean_weights(fundamental_equity)

trend_equity = extract_clean_weights(trend_equity)
trend_bond = extract_clean_weights(trend_bond)
trend_rates = extract_clean_weights(trend_rates)
trend_fx = extract_clean_weights(trend_fx)

macroRS_equity = extract_clean_weights(macroRS_equity)
macroRS_bond = extract_clean_weights(macroRS_bond)
macroRS_rates = extract_clean_weights(macroRS_rates)
macroRS_fx = extract_clean_weights(macroRS_fx)
macroIT_equity = extract_clean_weights(macroIT_equity)
macroIT_bond = extract_clean_weights(macroIT_bond)
macroIT_rates = extract_clean_weights(macroIT_rates)
macroIT_fx = extract_clean_weights(macroIT_fx)
macroMP_equity = extract_clean_weights(macroMP_equity)
macroMP_bond = extract_clean_weights(macroMP_bond)
macroMP_rates = extract_clean_weights(macroMP_rates)
macroMP_fx = extract_clean_weights(macroMP_fx)
macroBC_equity = extract_clean_weights(macroBC_equity)
macroBC_bond = extract_clean_weights(macroBC_bond)
macroBC_rates = extract_clean_weights(macroBC_rates)
macroBC_fx = extract_clean_weights(macroBC_fx)



### Asset Data

In [736]:
# --- Load Equity Price Data ---
equity_prices = pd.read_excel(
    "../Data_Ryan/Equity Data.xlsx",
    index_col = 0,
    parse_dates = True
)
equity_prices.index = pd.to_datetime(equity_prices.index)
equity_prices.index = equity_prices.index + pd.offsets.MonthEnd(0)

equity_returns = equity_prices.pct_change()
equity_returns

,US,AU,CH,JP,UK,EM,EU
Date,,,,,,,
1997-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-10-31,-0.031954,NaN,NaN,-0.069246,-0.053835,NaN,NaN
1997-11-30,0.033550,NaN,NaN,-0.071232,0.009611,NaN,NaN
1997-12-31,0.013571,NaN,NaN,-0.075596,0.029754,NaN,NaN
1998-01-31,0.008938,NaN,NaN,0.085984,0.044176,NaN,NaN
...,...,...,...,...,...,...,...
2025-06-30,0.047850,0.034649,0.008742,0.018397,0.017408,0.087457,0.024923
2025-07-31,0.019268,-0.002424,-0.029467,-0.011530,0.000891,0.009281,-0.026534
2025-08-31,0.015453,0.045088,0.049595,0.068686,0.032134,0.008553,0.028645


In [737]:
# --- Load Bond Futures ---
bond_futures = pd.read_excel(
    "../Data_Ryan/Bond Data.xlsx",
    index_col = 0,
    parse_dates = True
)
bond_futures.index = pd.to_datetime(bond_futures.index)
bond_futures.index = bond_futures.index + pd.offsets.MonthEnd(0)

bond_returns = bond_futures.pct_change()
bond_returns

,EU,JP,AU,US,CH,EM,UK
Dates,,,,,,,
1998-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-02-28,0.017379,0.006342,-0.000532,-0.009009,0.013870,NaN,-0.000851
1998-03-31,-0.014688,0.003317,0.002553,-0.002674,-0.009084,NaN,0.039250
1998-04-30,0.026085,0.008926,-0.001061,-0.000536,-0.013143,NaN,0.002005
1998-05-31,0.015326,0.017256,0.005684,0.005901,0.021489,NaN,-0.013159
...,...,...,...,...,...,...,...
2025-07-31,-0.032262,-0.007101,-0.022422,-0.009475,-0.015542,-0.009048,-0.044820
2025-08-31,0.023240,-0.003576,0.017618,0.012519,0.020001,0.005948,0.003818
2025-09-30,0.004901,-0.005493,0.010887,0.000139,0.021848,0.001495,0.000005


In [738]:
# --- Load Rates Futures ---
rates_futures = pd.read_excel(
    "../Data_Ryan/Interest Rates Data.xlsx",
    index_col = 0,
    parse_dates = True
)
rates_futures.index = pd.to_datetime(rates_futures.index)
rates_futures.index = rates_futures.index + pd.offsets.MonthEnd(0)

rates_returns = rates_futures.pct_change()
rates_returns

,CH,EU,AU,US,EM,UK,JP
Date,,,,,,,
1998-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-03-31,-0.002432,NaN,0.000844,0.000524,0.013047,NaN,0.001312
1998-04-30,-0.000509,NaN,-0.000730,-0.000209,0.013644,NaN,0.000302
1998-05-31,0.000000,NaN,0.000422,-0.000733,-0.014970,NaN,0.000554
...,...,...,...,...,...,...,...
2025-07-31,0.000000,-0.001019,-0.002281,0.000000,0.000000,-0.001840,0.000000
2025-08-31,0.000000,-0.000459,-0.000207,0.000000,-0.009700,-0.002765,0.000000
2025-09-30,0.000000,0.000051,-0.001759,0.000000,-0.004021,-0.001232,0.000000


In [739]:
# --- Load Currency Prices ---
fx_data = pd.read_excel(
    "../Data_Ryan/FX Data.xlsx",
    sheet_name = 'RETURNS',
    index_col = 0,
    parse_dates = True
)
fx_data.index = pd.to_datetime(fx_data.index)
fx_data.index = fx_data.index + pd.offsets.MonthEnd(0)
fx_returns = fx_data
fx_returns



,CH,EU,JP,UK,AU,EM
Date,,,,,,
1989-01-31,-0.062505,NaN,-0.044448,-0.028124,0.043550,NaN
1989-02-28,0.020885,NaN,0.025663,-0.002575,-0.095372,NaN
1989-03-31,-0.067754,NaN,-0.048472,-0.030821,0.027844,NaN
1989-04-30,-0.009532,NaN,-0.006180,0.004816,-0.025641,NaN
1989-05-31,-0.022300,NaN,-0.070709,-0.068691,-0.047106,NaN
...,...,...,...,...,...,...
2025-06-30,0.033185,0.036774,-0.003389,0.020141,0.022884,0.004021
2025-07-31,-0.027679,-0.033666,-0.048025,-0.038372,-0.024318,-0.009627
2025-08-31,0.010759,0.021636,0.021495,0.022178,0.017316,0.009796


### Riskfree Data

In [740]:
# --- Load Riskfree Rate ---
factors_data = pd.read_excel(
    "../Data_Ryan/Factors.xlsx",
    index_col = 0,
    parse_dates = True
)

factors_data.index = pd.to_datetime(factors_data.index, format='%Y%m')
factors_data.index = factors_data.index + pd.offsets.MonthEnd(0)
factors_data /= 100

riskfree = factors_data["RF"].resample('ME').last()
riskfree

/var/folders/37/zcyd0byn17lfrmxtt4_4k3s40000gn/T/ipykernel_94985/2112351923.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  factors_data = pd.read_excel(


1926-07-31    0.0022
1926-08-31    0.0025
1926-09-30    0.0023
1926-10-31    0.0032
1926-11-30    0.0031
               ...  
2025-06-30    0.0034
2025-07-31    0.0034
2025-08-31    0.0038
2025-09-30    0.0033
2025-10-31    0.0037
Freq: ME, Name: RF, Length: 1192, dtype: float64

### Benchmark Data

In [741]:
# --- Benchmark Data ---
benchmark_data = pd.read_excel(
    "../Data_Ryan/Benchmarks.xlsx",
    index_col = 0,
    parse_dates = True
)

benchmark_data.index = pd.to_datetime(benchmark_data.index)
benchmark_data = benchmark_data.resample('ME').last()


benchmark_return = benchmark_data[['MSCI World']].pct_change()
benchmark_return = benchmark_return.squeeze()
benchmark_return

Date
1986-12-31         NaN
1987-01-31         NaN
1987-02-28         NaN
1987-03-31         NaN
1987-04-30         NaN
                ...   
2025-07-31    0.013121
2025-08-31    0.026408
2025-09-30    0.032574
2025-10-31    0.020226
2025-11-30    0.003149
Freq: ME, Name: MSCI World, Length: 468, dtype: float64

### Factor Data

In [742]:
# --- Load Factors Data ---
famafrench_data = pd.read_csv(
    "../Data_Ryan/famafrench_factors.csv",
    index_col = 0,
    parse_dates = True
)

famafrench_data.index = pd.to_datetime(famafrench_data.index, format='%Y%m')
famafrench_data.index = famafrench_data.index + pd.offsets.MonthEnd(0)
famafrench_data.dropna(inplace=True)
famafrench_data

/var/folders/37/zcyd0byn17lfrmxtt4_4k3s40000gn/T/ipykernel_94985/3295830964.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  famafrench_data = pd.read_csv(


,MKT-RF,SMB,HML,RMW,CMA,UMD,BAB
DATE,,,,,,,
1980-01-31,0.0550,0.0188,0.0185,-0.0184,0.0189,0.0745,0.0695
1980-02-29,-0.0123,-0.0162,0.0059,-0.0095,0.0292,0.0789,-0.0132
1980-03-31,-0.1289,-0.0697,-0.0096,0.0182,-0.0105,-0.0958,-0.1181
1980-04-30,0.0396,0.0105,0.0103,-0.0218,0.0034,-0.0048,0.0574
1980-05-31,0.0526,0.0200,0.0038,0.0043,-0.0063,-0.0118,0.0618
...,...,...,...,...,...,...,...
2025-05-31,0.0606,-0.0072,-0.0288,0.0129,0.0251,0.0221,0.0256
2025-06-30,0.0486,-0.0002,-0.0160,-0.0320,0.0145,-0.0264,0.0527
2025-07-31,0.0198,-0.0015,-0.0127,-0.0029,-0.0208,-0.0096,0.0184


### Hedging Data

In [743]:
# Read the CSV
fx_predictions = pd.read_csv("../Data/FX_predictions.csv", index_col=0, parse_dates=["Date"])
fx_predictions.index = fx_predictions.index + pd.offsets.MonthEnd(0)

# Keep only Date, Region, Hedge Ratio
df_fx = fx_predictions[["Region", "Hedge_Ratio_Next_Month"]]

# Pivot to wide format
hedge_ratio = df_fx.pivot_table(
    index=df_fx.index,
    columns="Region",
    values="Hedge_Ratio_Next_Month"
)

if "GB" in hedge_ratio.columns:
    hedge_ratio = hedge_ratio.rename(columns={"GB": "UK"})

# Ensure proper column order (optional)
df_fhedge_ratiox_wide = hedge_ratio.sort_index(axis=1)

hedge_ratio.dropna(inplace=True)
hedge_ratio


Region,AU,CH,EM,EU,UK,JP
Date,,,,,,
2000-04-30,0.75,0.50,0.50,0.50,1.0,0.50
2000-05-31,1.00,0.75,0.50,0.75,1.0,0.50
2000-06-30,1.00,0.50,0.50,0.50,1.0,0.75
2000-07-31,1.00,0.50,0.50,0.50,1.0,0.75
2000-08-31,0.50,0.50,0.50,0.50,1.0,0.50
...,...,...,...,...,...,...
2025-06-30,1.00,0.50,0.50,0.50,0.5,0.00
2025-07-31,1.00,0.50,0.50,0.50,0.5,0.00
2025-08-31,0.50,0.50,0.50,0.75,0.0,0.00


In [744]:
hedge_ratio_short = 1 - hedge_ratio
hedge_ratio_short

Region,AU,CH,EM,EU,UK,JP
Date,,,,,,
2000-04-30,0.25,0.50,0.50,0.50,0.0,0.50
2000-05-31,0.00,0.25,0.50,0.25,0.0,0.50
2000-06-30,0.00,0.50,0.50,0.50,0.0,0.25
2000-07-31,0.00,0.50,0.50,0.50,0.0,0.25
2000-08-31,0.50,0.50,0.50,0.50,0.0,0.50
...,...,...,...,...,...,...
2025-06-30,0.00,0.50,0.50,0.50,0.5,1.00
2025-07-31,0.00,0.50,0.50,0.50,0.5,1.00
2025-08-31,0.50,0.50,0.50,0.25,1.0,1.00


In [745]:
# --- Load Currency Prices ---
hedge_return = pd.read_excel(
    "../Data_Ryan/FX Data.xlsx",
    sheet_name = 'HEDGE RETURN',
    index_col = 0,
    parse_dates = True
)
hedge_return.index = pd.to_datetime(hedge_return.index)
hedge_return.index = hedge_return.index + pd.offsets.MonthEnd(0)
hedge_returns = hedge_return
hedge_returns

,CH,EU,JP,UK,AU,EM
Date,,,,,,
1989-01-31,0.062505,0.000000,0.044448,0.028124,-0.043550,0.000000
1989-02-28,-0.020885,0.000000,-0.025663,0.002575,0.095372,0.000000
1989-03-31,0.067754,0.000000,0.048472,0.030821,-0.027844,0.000000
1989-04-30,0.009532,0.000000,0.006180,-0.004816,0.025641,0.000000
1989-05-31,0.022300,0.000000,0.070709,0.068691,0.047106,0.000000
...,...,...,...,...,...,...
2025-06-30,-0.033185,-0.036774,0.003389,-0.020141,-0.022884,-0.004021
2025-07-31,0.027679,0.033666,0.048025,0.038372,0.024318,0.009627
2025-08-31,-0.010759,-0.021636,-0.021495,-0.022178,-0.017316,-0.009796


## Combine Macro Strategies

In [746]:
macro_equity = combine_macro_weights(
    dfs=[
        macroRS_equity,
        macroIT_equity,
        macroMP_equity,
        macroBC_equity
    ],
    weights=[0.25, 0.25, 0.25, 0.25]
)
macro_bond = combine_macro_weights(
    dfs=[
        macroRS_bond,
        macroIT_bond,
        macroMP_bond,
        macroBC_bond
    ],
    weights=[0.25, 0.25, 0.25, 0.25]
)

macro_rates = combine_macro_weights(
    dfs=[
        macroRS_rates,
        macroIT_rates,
        macroMP_rates,
        macroBC_rates
    ],
    weights=[0.25, 0.25, 0.25, 0.25]
)

macro_fx = combine_macro_weights(
    dfs=[
        macroRS_fx,
        macroIT_fx,
        macroMP_fx,
        macroBC_fx
    ],
    weights=[0.25, 0.25, 0.25, 0.25]
)

# Combined Strategies

## 1. Trend + Macro

### Individual Backtests

In [747]:
bond_macro_trend = combine_two_weight_tables(trend_bond, macro_bond, 0.5, 0.5)

bond_macro_trend_results = pmp.run_cc_strategy_drift(
    weights      = bond_macro_trend,
    returns      = bond_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

In [748]:
equity_macro_trend = combine_two_weight_tables(trend_equity, macro_equity, 0.5, 0.5)

equity_macro_trend_results = pmp.run_cc_strategy_drift(
    weights      = equity_macro_trend,
    returns      = equity_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

In [749]:
fx_macro_trend = combine_two_weight_tables(trend_fx, macro_fx, 0.5, 0.5)

fx_macro_trend_results = pmp.run_cc_strategy_drift(
    weights      = fx_macro_trend,
    returns      = fx_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

In [750]:
rates_macro_trend = combine_two_weight_tables(trend_rates, macro_rates, 0.5, 0.5)

rates_macro_trend_results = pmp.run_cc_strategy_drift(
    weights      = rates_macro_trend,
    returns      = rates_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

### Combined

In [751]:
macro_trend = combine_strategy_results(
    dfs=[
        equity_macro_trend_results,
        bond_macro_trend_results,
        fx_macro_trend_results,
        rates_macro_trend_results
    ],
    weights=[0.25, 0.25, 0.25, 0.25]
)

macro_trend

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EM,w_EU,w_JP,w_UK,w_US
Date,,,,,,,,,,,,,
1999-12-31,0.002405,0.002637,0.080969,1.160480,0.000232,0.0044,0.297822,-0.270208,0.136560,-0.135658,0.242737,0.046524,-0.315140
2000-01-31,0.007054,0.007270,-0.057716,1.081656,0.000216,0.0041,-0.313497,-0.282196,0.272696,0.001598,0.241647,0.040409,0.046612
2000-02-29,0.000760,0.000908,0.002763,0.740696,0.000148,0.0043,-0.166808,0.144220,0.260390,-0.000520,0.204099,-0.127050,-0.313423
2000-03-31,-0.001814,-0.001689,0.069919,0.622311,0.000124,0.0047,-0.102084,0.033141,0.176279,-0.253548,0.427341,-0.024861,-0.257959
2000-04-30,0.005503,0.005572,-0.041894,0.347064,0.000069,0.0046,-0.185889,0.041158,0.242944,-0.109321,0.246598,-0.083499,-0.146420
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-31,0.002851,0.002939,0.059891,0.436284,0.000087,0.0038,-0.299433,-0.079183,0.258055,-0.224485,-0.000167,0.292417,0.055734
2025-06-30,0.000492,0.000632,0.043488,0.699207,0.000140,0.0034,-0.204118,0.158554,0.237421,-0.260427,-0.131003,0.082540,0.117665
2025-07-31,-0.006279,-0.006019,0.013121,1.299789,0.000260,0.0034,-0.044335,-0.149071,-0.129773,0.287710,-0.066312,0.184176,-0.088414


### Hedged Performance

In [752]:
macro_trend_hedged = apply_fx_hedging(
    backtest_df=macro_trend,
    hedge_ratio=hedge_ratio,
    hedge_ratio_short=hedge_ratio_short,
    hedge_returns=hedge_returns
)

In [753]:
macro_trend_hedged.dropna(inplace=True)
macro_trend_hedged

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EM,w_EU,w_JP,w_UK,w_US
Date,,,,,,,,,,,,,
2000-04-30,0.008588,0.005572,-0.041894,0.347064,0.000069,0.0046,-0.185889,0.041158,0.242944,-0.109321,0.246598,-0.083499,-0.146420
2000-05-31,0.002472,-0.004983,-0.024084,0.629010,0.000126,0.0050,0.235485,-0.319582,0.225989,-0.212236,0.312964,-0.100740,-0.146863
2000-06-30,-0.000413,-0.001663,0.034036,0.607815,0.000122,0.0040,-0.265019,-0.319302,0.201693,-0.014278,0.343188,0.023580,0.028476
2000-07-31,0.005382,0.001621,-0.028304,0.609512,0.000122,0.0048,-0.185547,-0.211011,0.244209,-0.206919,0.343977,-0.086426,0.103337
2000-08-31,-0.001443,0.010409,0.033090,0.583696,0.000117,0.0050,-0.114704,-0.208207,0.110027,-0.182475,0.273492,-0.164766,0.297042
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-31,0.000584,0.002939,0.059891,0.436284,0.000087,0.0038,-0.299433,-0.079183,0.258055,-0.224485,-0.000167,0.292417,0.055734
2025-06-30,0.000897,0.000632,0.043488,0.699207,0.000140,0.0034,-0.204118,0.158554,0.237421,-0.260427,-0.131003,0.082540,0.117665
2025-07-31,-0.003775,-0.006019,0.013121,1.299789,0.000260,0.0034,-0.044335,-0.149071,-0.129773,0.287710,-0.066312,0.184176,-0.088414


In [754]:
macro_trend_hedged.dropna(inplace=True)
pmp.run_perf_summary_benchmark_vs_strategy(macro_trend_hedged, alreadyXs=True)

,Benchmark,Strategy
Arithm Avg Total Return,7.9101,1.7878
Arithm Avg Xs Return,6.1066,-0.0157
Std Xs Returns,15.4412,2.7136
Sharpe Arithmetic,0.3955,-0.0058
Geom Avg Total Return,6.9195,1.7639
Geom Avg Xs Return,5.1026,-0.0530
Sharpe Geometric,0.3305,-0.0195
Min Xs Return,-19.0140,-3.4081
Max Xs Return,12.8184,2.8453
Skewness,-0.5731,0.1231


In [755]:
pmp.run_factor_regression(macro_trend_hedged, famafrench_data, alreadyXs=True)

,Strategy
Arithm Avg Total Return,1.7878
Arithm Avg Xs Return,-0.0157
Std Xs Returns,2.7136
Sharpe Arithmetic,-0.0058
Geom Avg Total Return,1.7639
Geom Avg Xs Return,-0.0530
Sharpe Geometric,-0.0195
Min Xs Return,-3.4081
Max Xs Return,2.8453
Skewness,0.1231


### Unhedged Performance

In [756]:
macro_trend.dropna(inplace=True)
pmp.run_perf_summary_benchmark_vs_strategy(macro_trend, alreadyXs=True)

,Benchmark,Strategy
Arithm Avg Total Return,8.1794,2.2685
Arithm Avg Xs Return,6.3314,0.4205
Std Xs Returns,15.5076,2.3900
Sharpe Arithmetic,0.4083,0.1759
Geom Avg Total Return,7.1949,2.2612
Geom Avg Xs Return,5.3328,0.3991
Sharpe Geometric,0.3439,0.1670
Min Xs Return,-19.0140,-2.7562
Max Xs Return,12.8184,3.2527
Skewness,-0.5608,0.1188


In [757]:
pmp.run_factor_regression(macro_trend, famafrench_data, alreadyXs=True)

,Strategy
Arithm Avg Total Return,2.2685
Arithm Avg Xs Return,0.4205
Std Xs Returns,2.3900
Sharpe Arithmetic,0.1759
Geom Avg Total Return,2.2612
Geom Avg Xs Return,0.3991
Sharpe Geometric,0.1670
Min Xs Return,-2.7562
Max Xs Return,3.2527
Skewness,0.1188


## 2.  Trend + Fundamental

### Individual Backtests

In [758]:
fundamental_equity_results = pmp.run_cc_strategy_drift(
    weights      = fundamental_equity,
    returns      = equity_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

In [759]:
trend_equity_results = pmp.run_cc_strategy_drift(
    weights      = trend_equity,
    returns      = equity_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

In [760]:
trend_bond_results = pmp.run_cc_strategy_drift(
    weights      = trend_bond,
    returns      = bond_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

In [761]:
trend_fx_results = pmp.run_cc_strategy_drift(
    weights      = trend_fx,
    returns      = fx_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

In [762]:
trend_rates_results = pmp.run_cc_strategy_drift(
    weights      = trend_rates,
    returns      = rates_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

### Combined

In [763]:
fundamental_trend = combine_strategy_results(
    dfs=[
        fundamental_equity_results,
        trend_equity_results,
        trend_bond_results,
        trend_fx_results,
        trend_rates_results,
    ],
    weights=[0.50, 0.125, 0.125, 0.125, 0.125]
)

fundamental_trend

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EM,w_EU,w_JP,w_UK,w_US
Date,,,,,,,,,,,,,
2000-11-30,0.003921,0.004105,-0.060660,0.918728,0.000184,0.0051,-0.048151,0.144833,0.243112,-0.318445,-0.003637,-0.426885,0.258962
2000-12-31,0.004758,0.004948,0.015919,0.948900,0.000190,0.0050,-0.119234,0.061032,0.188992,-0.216891,-0.291526,-0.089906,0.266538
2001-01-31,0.002612,0.002809,0.019408,0.988711,0.000198,0.0054,0.272116,-0.312263,0.294825,-0.208797,0.046473,0.180525,-0.132656
2001-02-28,-0.009343,-0.009244,-0.084288,0.493037,0.000099,0.0038,0.260574,-0.193855,0.264852,-0.101982,-0.091885,0.049896,0.038620
2001-03-31,-0.002271,-0.002156,-0.065527,0.576224,0.000115,0.0042,0.210376,-0.064450,0.061171,-0.062129,-0.307101,0.195849,0.013043
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-31,0.017900,0.018101,0.059891,1.006873,0.000201,0.0038,-0.134767,-0.279961,-0.005527,-0.323626,-0.056256,0.324542,0.475538
2025-06-30,-0.005861,-0.005682,0.043488,0.894617,0.000179,0.0034,-0.158886,0.029198,-0.085001,-0.377704,-0.100448,0.175898,0.553989
2025-07-31,0.009778,0.010084,0.013121,1.529197,0.000306,0.0034,0.080008,-0.259660,-0.391412,0.120446,-0.227117,0.304528,0.382956


### Hedged Performance

In [764]:
fundamental_trend_hedged = apply_fx_hedging(
    backtest_df=fundamental_trend,
    hedge_ratio=hedge_ratio,
    hedge_ratio_short=hedge_ratio_short,
    hedge_returns=hedge_returns
)


In [765]:
fundamental_trend_hedged.dropna(inplace=True)
fundamental_trend_hedged

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EM,w_EU,w_JP,w_UK,w_US
Date,,,,,,,,,,,,,
2000-11-30,-0.002622,0.004105,-0.060660,0.918728,0.000184,0.0051,-0.048151,0.144833,0.243112,-0.318445,-0.003637,-0.426885,0.258962
2000-12-31,-0.003502,0.004948,0.015919,0.948900,0.000190,0.0050,-0.119234,0.061032,0.188992,-0.216891,-0.291526,-0.089906,0.266538
2001-01-31,0.005051,0.002809,0.019408,0.988711,0.000198,0.0054,0.272116,-0.312263,0.294825,-0.208797,0.046473,0.180525,-0.132656
2001-02-28,-0.012753,-0.009244,-0.084288,0.493037,0.000099,0.0038,0.260574,-0.193855,0.264852,-0.101982,-0.091885,0.049896,0.038620
2001-03-31,-0.015299,-0.002156,-0.065527,0.576224,0.000115,0.0042,0.210376,-0.064450,0.061171,-0.062129,-0.307101,0.195849,0.013043
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-31,0.015770,0.018101,0.059891,1.006873,0.000201,0.0038,-0.134767,-0.279961,-0.005527,-0.323626,-0.056256,0.324542,0.475538
2025-06-30,-0.001342,-0.005682,0.043488,0.894617,0.000179,0.0034,-0.158886,0.029198,-0.085001,-0.377704,-0.100448,0.175898,0.553989
2025-07-31,0.003209,0.010084,0.013121,1.529197,0.000306,0.0034,0.080008,-0.259660,-0.391412,0.120446,-0.227117,0.304528,0.382956


In [766]:
fundamental_trend_hedged.dropna(inplace=True)
pmp.run_perf_summary_benchmark_vs_strategy(fundamental_trend_hedged, alreadyXs=True)

,Benchmark,Strategy
Arithm Avg Total Return,8.4858,2.9940
Arithm Avg Xs Return,6.7769,1.2851
Std Xs Returns,15.4752,5.6861
Sharpe Arithmetic,0.4379,0.2260
Geom Avg Total Return,7.5248,2.8707
Geom Avg Xs Return,5.8039,1.1498
Sharpe Geometric,0.3750,0.2022
Min Xs Return,-19.0140,-4.8127
Max Xs Return,12.8184,5.1558
Skewness,-0.6022,0.2311


In [767]:
pmp.run_factor_regression(fundamental_trend_hedged, famafrench_data, alreadyXs=True)

,Strategy
Arithm Avg Total Return,2.9940
Arithm Avg Xs Return,1.2851
Std Xs Returns,5.6861
Sharpe Arithmetic,0.2260
Geom Avg Total Return,2.8707
Geom Avg Xs Return,1.1498
Sharpe Geometric,0.2022
Min Xs Return,-4.8127
Max Xs Return,5.1558
Skewness,0.2311


### Unhedged Performance

In [768]:
fundamental_trend.dropna(inplace=True)
pmp.run_perf_summary_benchmark_vs_strategy(fundamental_trend, alreadyXs=True)

,Benchmark,Strategy
Arithm Avg Total Return,8.4858,3.9061
Arithm Avg Xs Return,6.7769,2.1972
Std Xs Returns,15.4752,5.3087
Sharpe Arithmetic,0.4379,0.4139
Geom Avg Total Return,7.5248,3.8317
Geom Avg Xs Return,5.8039,2.1108
Sharpe Geometric,0.3750,0.3976
Min Xs Return,-19.0140,-3.4203
Max Xs Return,12.8184,6.2809
Skewness,-0.6022,0.4178


In [769]:
pmp.run_factor_regression(fundamental_trend, famafrench_data, alreadyXs=True)

,Strategy
Arithm Avg Total Return,3.9061
Arithm Avg Xs Return,2.1972
Std Xs Returns,5.3087
Sharpe Arithmetic,0.4139
Geom Avg Total Return,3.8317
Geom Avg Xs Return,2.1108
Sharpe Geometric,0.3976
Min Xs Return,-3.4203
Max Xs Return,6.2809
Skewness,0.4178


## 3. Fundamental + Macro

### Individual Backtests

In [770]:
fundamental_equity_results = pmp.run_cc_strategy_drift(
    weights      = fundamental_equity,
    returns      = equity_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

In [771]:
macro_equity_results = pmp.run_cc_strategy_drift(
    weights      = macro_equity,
    returns      = equity_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

In [772]:
macro_bond_results = pmp.run_cc_strategy_drift(
    weights      = macro_bond,
    returns      = bond_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

In [773]:
macro_fx_results = pmp.run_cc_strategy_drift(
    weights      = macro_fx,
    returns      = fx_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

In [774]:
macro_rates_results = pmp.run_cc_strategy_drift(
    weights      = macro_rates,
    returns      = rates_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

### Combined

In [775]:
fundamental_macro = combine_strategy_results(
    dfs=[
        fundamental_equity_results,
        macro_equity_results,
        macro_bond_results,
        macro_fx_results,
        macro_rates_results,
    ],
    weights=[0.50, 0.125, 0.125, 0.125, 0.125]
)

fundamental_macro

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EM,w_EU,w_JP,w_UK,w_US
Date,,,,,,,,,,,,,
2003-02-28,-0.000129,-0.000108,-0.017101,0.108606,0.000022,0.0009,0.000000,0.072254,0.000000,-0.023431,0.176634,-0.144116,0.142252
2003-03-31,-0.006064,-0.005983,-0.002623,0.406273,0.000081,0.0010,0.000000,-0.090451,0.000000,-0.062703,0.225096,0.303787,0.041124
2003-04-30,0.005423,0.005519,0.089437,0.478156,0.000096,0.0010,0.000000,0.196165,0.000000,-0.014800,0.103342,-0.050509,-0.140052
2003-05-31,-0.007650,-0.007575,0.057688,0.379322,0.000076,0.0009,0.000000,-0.018065,0.000000,0.044667,0.339099,-0.309365,-0.180356
2003-06-30,0.005832,0.005943,0.017762,0.556157,0.000111,0.0010,0.000000,-0.234755,0.000000,0.053665,0.109213,0.277374,-0.244156
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-31,0.015048,0.015162,0.059891,0.570589,0.000114,0.0038,0.164666,-0.200778,-0.263582,-0.099141,-0.056089,0.032125,0.419805
2025-06-30,-0.006353,-0.006314,0.043488,0.195409,0.000039,0.0034,0.045231,-0.129357,-0.322421,-0.117277,0.030555,0.093358,0.436324
2025-07-31,0.016057,0.016103,0.013121,0.229408,0.000046,0.0034,0.124343,-0.110589,-0.261640,-0.167264,-0.160805,0.120353,0.471370


### Hedged Performance

In [776]:
fundamental_macro_hedged = apply_fx_hedging(
    backtest_df=fundamental_macro,
    hedge_ratio=hedge_ratio,
    hedge_ratio_short=hedge_ratio_short,
    hedge_returns=hedge_returns
)


In [777]:
fundamental_macro_hedged.dropna(inplace=True)
fundamental_macro_hedged

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EM,w_EU,w_JP,w_UK,w_US
Date,,,,,,,,,,,,,
2003-02-28,-0.001758,-0.000108,-0.017101,0.108606,0.000022,0.0009,0.000000,0.072254,0.000000,-0.023431,0.176634,-0.144116,0.142252
2003-03-31,-0.008493,-0.005983,-0.002623,0.406273,0.000081,0.0010,0.000000,-0.090451,0.000000,-0.062703,0.225096,0.303787,0.041124
2003-04-30,0.006008,0.005519,0.089437,0.478156,0.000096,0.0010,0.000000,0.196165,0.000000,-0.014800,0.103342,-0.050509,-0.140052
2003-05-31,-0.008680,-0.007575,0.057688,0.379322,0.000076,0.0009,0.000000,-0.018065,0.000000,0.044667,0.339099,-0.309365,-0.180356
2003-06-30,0.000442,0.005943,0.017762,0.556157,0.000111,0.0010,0.000000,-0.234755,0.000000,0.053665,0.109213,0.277374,-0.244156
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-31,0.014492,0.015162,0.059891,0.570589,0.000114,0.0038,0.164666,-0.200778,-0.263582,-0.099141,-0.056089,0.032125,0.419805
2025-06-30,-0.003378,-0.006314,0.043488,0.195409,0.000039,0.0034,0.045231,-0.129357,-0.322421,-0.117277,0.030555,0.093358,0.436324
2025-07-31,0.008061,0.016103,0.013121,0.229408,0.000046,0.0034,0.124343,-0.110589,-0.261640,-0.167264,-0.160805,0.120353,0.471370


In [778]:
fundamental_macro_hedged.dropna(inplace=True)
pmp.run_perf_summary_benchmark_vs_strategy(fundamental_macro_hedged, alreadyXs=True)

,Benchmark,Strategy
Arithm Avg Total Return,11.2481,2.9203
Arithm Avg Xs Return,9.6559,1.3281
Std Xs Returns,15.0151,5.0233
Sharpe Arithmetic,0.6431,0.2644
Geom Avg Total Return,10.5895,2.8311
Geom Avg Xs Return,8.9871,1.2287
Sharpe Geometric,0.5985,0.2446
Min Xs Return,-19.0140,-4.9983
Max Xs Return,12.8184,4.2652
Skewness,-0.6694,0.0692


In [779]:
pmp.run_factor_regression(fundamental_macro_hedged, famafrench_data, alreadyXs=True)

,Strategy
Arithm Avg Total Return,2.9203
Arithm Avg Xs Return,1.3281
Std Xs Returns,5.0233
Sharpe Arithmetic,0.2644
Geom Avg Total Return,2.8311
Geom Avg Xs Return,1.2287
Sharpe Geometric,0.2446
Min Xs Return,-4.9983
Max Xs Return,4.2652
Skewness,0.0692


### Unhedged Performance

In [780]:
fundamental_macro.dropna(inplace=True)
pmp.run_perf_summary_benchmark_vs_strategy(fundamental_macro, alreadyXs=True)

,Benchmark,Strategy
Arithm Avg Total Return,11.2481,3.2923
Arithm Avg Xs Return,9.6559,1.7001
Std Xs Returns,15.0151,4.5694
Sharpe Arithmetic,0.6431,0.3721
Geom Avg Total Return,10.5895,3.2354
Geom Avg Xs Return,8.9871,1.6329
Sharpe Geometric,0.5985,0.3574
Min Xs Return,-19.0140,-3.4367
Max Xs Return,12.8184,4.4502
Skewness,-0.6694,0.1779


In [781]:
pmp.run_factor_regression(fundamental_macro, famafrench_data, alreadyXs=True)

,Strategy
Arithm Avg Total Return,3.2923
Arithm Avg Xs Return,1.7001
Std Xs Returns,4.5694
Sharpe Arithmetic,0.3721
Geom Avg Total Return,3.2354
Geom Avg Xs Return,1.6329
Sharpe Geometric,0.3574
Min Xs Return,-3.4367
Max Xs Return,4.4502
Skewness,0.1779


## 4. Fundamental + Macro + Trend

### Individual Backtests

#### Macro + Trend (Weight = 2/3)

In [782]:
macro_trend # see above

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EM,w_EU,w_JP,w_UK,w_US
Date,,,,,,,,,,,,,
1999-12-31,0.002405,0.002637,0.080969,1.160480,0.000232,0.0044,0.297822,-0.270208,0.136560,-0.135658,0.242737,0.046524,-0.315140
2000-01-31,0.007054,0.007270,-0.057716,1.081656,0.000216,0.0041,-0.313497,-0.282196,0.272696,0.001598,0.241647,0.040409,0.046612
2000-02-29,0.000760,0.000908,0.002763,0.740696,0.000148,0.0043,-0.166808,0.144220,0.260390,-0.000520,0.204099,-0.127050,-0.313423
2000-03-31,-0.001814,-0.001689,0.069919,0.622311,0.000124,0.0047,-0.102084,0.033141,0.176279,-0.253548,0.427341,-0.024861,-0.257959
2000-04-30,0.005503,0.005572,-0.041894,0.347064,0.000069,0.0046,-0.185889,0.041158,0.242944,-0.109321,0.246598,-0.083499,-0.146420
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-31,0.002851,0.002939,0.059891,0.436284,0.000087,0.0038,-0.299433,-0.079183,0.258055,-0.224485,-0.000167,0.292417,0.055734
2025-06-30,0.000492,0.000632,0.043488,0.699207,0.000140,0.0034,-0.204118,0.158554,0.237421,-0.260427,-0.131003,0.082540,0.117665
2025-07-31,-0.006279,-0.006019,0.013121,1.299789,0.000260,0.0034,-0.044335,-0.149071,-0.129773,0.287710,-0.066312,0.184176,-0.088414


#### Fundamental (Weight = 1/3)

In [783]:
fundamental_equity_results

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EM,w_EU,w_UK,w_JP,w_US
Date,,,,,,,,,,,,,
2000-11-30,0.027399,0.027399,-0.060660,0.000000,0.000000,0.0051,0.000000,0.000000,0.000000,0.000000,-0.623386,0.162907,0.179245
2000-12-31,0.018854,0.018962,0.015919,0.537498,0.000107,0.0050,0.000000,0.000000,0.000000,0.000000,-0.159189,-0.261421,0.027687
2001-01-31,-0.004059,-0.003977,0.019408,0.411558,0.000082,0.0054,0.000000,0.000000,0.000000,0.000000,0.295443,0.026827,-0.051418
2001-02-28,-0.030587,-0.030567,-0.084288,0.100037,0.000020,0.0038,0.000000,0.000000,0.000000,0.000000,0.312105,0.144955,-0.016702
2001-03-31,-0.006546,-0.006497,-0.065527,0.246314,0.000049,0.0042,0.000000,0.000000,0.000000,0.000000,0.360967,0.003137,-0.272769
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,-0.012707,-0.012629,0.043488,0.390818,0.000078,0.0034,0.090462,-0.258713,-0.644842,-0.234554,0.186717,0.061110,0.872648
2025-07-31,0.032114,0.032205,0.013121,0.458816,0.000092,0.0034,0.248686,-0.221178,-0.523279,-0.334528,0.240705,-0.321610,0.942740
2025-08-31,0.017252,0.017447,0.026408,0.974642,0.000195,0.0038,0.146615,0.093660,-0.384734,-0.226550,0.257767,0.104881,0.098647


### Combined

In [784]:
combined = combine_strategy_results(
    dfs=[
        fundamental_equity_results,
        macro_trend
    ],
    weights=[0.25, 0.75]
)

combined

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EM,w_EU,w_JP,w_UK,w_US
Date,,,,,,,,,,,,,
2000-11-30,-0.000484,-0.000346,-0.060660,0.689046,0.000138,0.0051,-0.036113,0.108625,0.182334,-0.238834,-0.023091,-0.242241,0.171816
2000-12-31,0.001212,0.001341,0.015919,0.644488,0.000129,0.0050,-0.089425,0.045774,0.141744,-0.162668,-0.185967,-0.047531,0.196443
2001-01-31,0.002466,0.002604,0.019408,0.690089,0.000138,0.0054,0.204087,-0.234197,0.221119,-0.156598,0.031501,0.098463,-0.093065
2001-02-28,-0.003184,-0.003112,-0.084288,0.357273,0.000071,0.0038,0.195431,-0.145391,0.198639,-0.076487,-0.087034,-0.001591,0.031052
2001-03-31,-0.000885,-0.000805,-0.065527,0.401379,0.000080,0.0042,0.157782,-0.048337,0.045879,-0.046597,-0.230718,0.101766,0.043879
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-31,0.009663,0.009785,0.059891,0.612508,0.000123,0.0038,-0.142242,-0.159776,0.061750,-0.217934,-0.028170,0.235375,0.251703
2025-06-30,-0.002808,-0.002683,0.043488,0.622110,0.000124,0.0034,-0.130473,0.054237,0.016855,-0.253959,-0.082975,0.108584,0.306411
2025-07-31,0.003319,0.003537,0.013121,1.089545,0.000218,0.0034,0.028921,-0.167098,-0.228149,0.132151,-0.130137,0.198308,0.169374


### Hedged Performance

In [785]:
combined_hedged = apply_fx_hedging(
    backtest_df=combined,
    hedge_ratio=hedge_ratio,
    hedge_ratio_short=hedge_ratio_short,
    hedge_returns=hedge_returns
)


In [786]:
combined_hedged.dropna(inplace=True)
combined_hedged

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EM,w_EU,w_JP,w_UK,w_US
Date,,,,,,,,,,,,,
2000-11-30,-0.004315,-0.000346,-0.060660,0.689046,0.000138,0.0051,-0.036113,0.108625,0.182334,-0.238834,-0.023091,-0.242241,0.171816
2000-12-31,-0.004139,0.001341,0.015919,0.644488,0.000129,0.0050,-0.089425,0.045774,0.141744,-0.162668,-0.185967,-0.047531,0.196443
2001-01-31,0.003933,0.002604,0.019408,0.690089,0.000138,0.0054,0.204087,-0.234197,0.221119,-0.156598,0.031501,0.098463,-0.093065
2001-02-28,-0.005952,-0.003112,-0.084288,0.357273,0.000071,0.0038,0.195431,-0.145391,0.198639,-0.076487,-0.087034,-0.001591,0.031052
2001-03-31,-0.010671,-0.000805,-0.065527,0.401379,0.000080,0.0042,0.157782,-0.048337,0.045879,-0.046597,-0.230718,0.101766,0.043879
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-31,0.008031,0.009785,0.059891,0.612508,0.000123,0.0038,-0.142242,-0.159776,0.061750,-0.217934,-0.028170,0.235375,0.251703
2025-06-30,-0.000447,-0.002683,0.043488,0.622110,0.000124,0.0034,-0.130473,0.054237,0.016855,-0.253959,-0.082975,0.108584,0.306411
2025-07-31,0.000391,0.003537,0.013121,1.089545,0.000218,0.0034,0.028921,-0.167098,-0.228149,0.132151,-0.130137,0.198308,0.169374


In [787]:
combined_hedged.dropna(inplace=True)
pmp.run_perf_summary_benchmark_vs_strategy(combined_hedged, alreadyXs=True)

,Benchmark,Strategy
Arithm Avg Total Return,8.4858,2.2927
Arithm Avg Xs Return,6.7769,0.5838
Std Xs Returns,15.4752,3.2488
Sharpe Arithmetic,0.4379,0.1797
Geom Avg Total Return,7.5248,2.2628
Geom Avg Xs Return,5.8039,0.5419
Sharpe Geometric,0.3750,0.1668
Min Xs Return,-19.0140,-2.6144
Max Xs Return,12.8184,3.2892
Skewness,-0.6022,0.3049


In [789]:
pmp.run_factor_regression(combined_hedged, famafrench_data, alreadyXs=True)

,Strategy
Arithm Avg Total Return,2.2927
Arithm Avg Xs Return,0.5838
Std Xs Returns,3.2488
Sharpe Arithmetic,0.1797
Geom Avg Total Return,2.2628
Geom Avg Xs Return,0.5419
Sharpe Geometric,0.1668
Min Xs Return,-2.6144
Max Xs Return,3.2892
Skewness,0.3049


### Unhedged Performance

In [790]:
combined.dropna(inplace=True)
pmp.run_perf_summary_benchmark_vs_strategy(combined, alreadyXs=True)

,Benchmark,Strategy
Arithm Avg Total Return,8.4858,2.8953
Arithm Avg Xs Return,6.7769,1.1864
Std Xs Returns,15.4752,3.0025
Sharpe Arithmetic,0.4379,0.3951
Geom Avg Total Return,7.5248,2.8870
Geom Avg Xs Return,5.8039,1.1661
Sharpe Geometric,0.3750,0.3884
Min Xs Return,-19.0140,-1.9419
Max Xs Return,12.8184,3.5982
Skewness,-0.6022,0.4678


In [791]:
pmp.run_factor_regression(combined, famafrench_data, alreadyXs=True)

,Strategy
Arithm Avg Total Return,2.8953
Arithm Avg Xs Return,1.1864
Std Xs Returns,3.0025
Sharpe Arithmetic,0.3951
Geom Avg Total Return,2.8870
Geom Avg Xs Return,1.1661
Sharpe Geometric,0.3884
Min Xs Return,-1.9419
Max Xs Return,3.5982
Skewness,0.4678
